In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [2]:
import sys
import os
root_dir = "/gdrive/MyDrive/강의자료 모음/[2023-01] 실습 자료/12-2) Pretrained Language Model: PART II"
sys.path.append(root_dir)

In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 116.5 MB/s eta 0:00:00


In [4]:
from IPython.display import Image
Image(os.path.join(root_dir, "TitleGeneration.JPG"))

FileNotFoundError: ignored

FileNotFoundError: ignored

<IPython.core.display.Image object>

In [5]:
import logging

import os
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader, RandomSampler
from transformers import AdamW
import random
import numpy as np

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

def read_data(file_path):
    datas = []
    with open(file_path, "r", encoding="utf8") as infile:
        for line in infile:
          # 입력 데이터를 \t을 기준으로 분리
          pieces = line.strip().split("\t")
          article, title = pieces[0], pieces[1]
          datas.append((article, title))

    return datas


def convert_data2feature(datas, max_length, max_dec_length, tokenizer):
    input_ids_features, attention_mask_features, decoder_input_features, decoder_attention_mask_features, label_features = [], [], [], [], []

    for article, title in tqdm(datas, desc="convert_data2feature"):
        # tokenizer를 사용하여 입력 문장을 word piece 단위로 분리
        tokenized_article = tokenizer.tokenize(article)
        tokenized_title= tokenizer.tokenize(title)

        # word piece들을 대응하는 index로 치환
        input_ids = tokenizer.convert_tokens_to_ids(tokenized_article)
        # padding을 제외한 실제 데이터 정보를 반영해주기 위한 attention mask
        attention_mask = [1] * len(input_ids)
        # 각 문장을 구분하기 위한 정보를 반영해주기 위한 token type

        decoder_input = tokenizer.convert_tokens_to_ids(['<s>'] + tokenized_title)
        decoder_attention_mask = [1] * len(decoder_input)
        label = tokenizer.convert_tokens_to_ids(tokenized_title + ['</s>'])

        # padding 생성
        padding = [tokenizer.convert_tokens_to_ids(tokenizer.pad_token)] * (max_length - len(input_ids))
        input_ids += padding
        attention_mask += padding

        padding = [tokenizer.convert_tokens_to_ids(tokenizer.pad_token)] * (max_dec_length - len(decoder_input))
        decoder_input += padding
        decoder_attention_mask += padding
        label += padding

        # 변환한 데이터를 각 리스트에 저장
        input_ids_features.append(input_ids[:max_length])
        attention_mask_features.append(attention_mask[:max_length])
        decoder_input_features.append(decoder_input[:max_dec_length])
        decoder_attention_mask_features.append(decoder_attention_mask[:max_dec_length])  #####
        label_features.append(label[:max_dec_length])

    # 변환한 데이터를 Tensor 객체에 담아 반환
    input_ids_features = torch.tensor(input_ids_features, dtype=torch.long)
    attention_mask_features = torch.tensor(attention_mask_features, dtype=torch.long)
    decoder_input_features = torch.tensor(decoder_input_features, dtype=torch.long)
    decoder_attention_mask_features = torch.tensor(decoder_attention_mask_features, dtype=torch.long)
    label_features = torch.tensor(label_features, dtype=torch.long)

    return input_ids_features, attention_mask_features, decoder_input_features, decoder_attention_mask_features, label_features

def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [6]:
logger = logging.getLogger(__name__)
from torch.utils.data import (DataLoader, TensorDataset, RandomSampler, SequentialSampler)
from transformers import BartForConditionalGeneration
from transformers import PreTrainedTokenizerFast
def train(config):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(config["pretrained_model_name_or_path"])
    model = BartForConditionalGeneration.from_pretrained(config["pretrained_model_name_or_path"]).cuda()
    """ Train the model """
    # 학습 및 평가 데이터 읽기
    train_datas = read_data(config["train_data_path"])
    test_datas = read_data(config["test_data_path"])

    # 입력 데이터 전처리
    train_input_ids_features, train_attention_mask_features, train_decoder_input_features, train_decoder_attention_mask_features, train_label_features = \
        convert_data2feature(train_datas, config["max_length"],config["max_dec_length"], tokenizer)
    test_input_ids_features, test_attention_mask_features, test_decoder_input_features, test_decoder_attention_mask_features, test_label_features = \
        convert_data2feature(test_datas, config["max_length"], config["max_dec_length"], tokenizer)

    # 학습 데이터를 batch 단위로 추출하기 위한 DataLoader 객체 생성
    train_features = TensorDataset(train_input_ids_features, train_attention_mask_features, train_decoder_input_features, train_decoder_attention_mask_features, train_label_features)
    train_dataloader = DataLoader(train_features, sampler=RandomSampler(train_features),
                                  batch_size=config["batch_size"])

    # 평가 데이터를 batch 단위로 추출하기 위한 DataLoader 객체 생성
    test_features = TensorDataset(test_input_ids_features, test_attention_mask_features, test_decoder_input_features, test_decoder_attention_mask_features, test_label_features)
    test_dataloader = DataLoader(test_features, sampler=SequentialSampler(test_features),
                                 batch_size=config["batch_size"])

    # 모델 학습을 위한 optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
    global_step = 1

    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()

    set_seed(config["seed"])

    for epoch in range(config["epoch"]):
        for step, batch in enumerate(train_dataloader):
            # Skip past any already trained steps if resuming training
            model.train()
            batch = tuple(t.cuda() for t in batch)
            outputs = model(input_ids=batch[0],
                              attention_mask=batch[1],
                              decoder_input_ids=batch[2],
                              decoder_attention_mask=batch[3],
                              labels=batch[4],
                                return_dict=True)

            loss = outputs["loss"]

            loss.backward()
            if (global_step+1) % 50 == 0:
                print("{} Processed.. Total Loss : {}".format(global_step+1, loss.item()))

            tr_loss += loss.item()

            optimizer.step()
            model.zero_grad()
            global_step += 1

            # Save model checkpoint
            if global_step % 500 == 0:
                evaluate(config, model, tokenizer, test_dataloader)

                output_dir = os.path.join(config["output_dir_path"], "checkpoint-{}".format(global_step))
                print("Model Save in {}".format(output_dir))
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)

                # Take care of distributed/parallel training
                model_to_save = model.module if hasattr(model, "module") else model
                model_to_save.save_pretrained(output_dir)

    return global_step, tr_loss / global_step

def evaluate(config, model, tokenizer, test_dataloader):
    model.eval()
    for batch in tqdm(test_dataloader):
        batch = tuple(t.cuda() for t in batch)

        dec_outputs = model.generate(input_ids = batch[0],
                                     attention_mask=batch[1],
                                     max_length=config["max_dec_length"],
                                     eos_token_id=1,
                                     do_sample=False,
                                     bad_words_ids=[[5]]
                                    )

        batch_size = batch[0].size()[0]

        dec_outputs = dec_outputs.tolist()
        dec_labels = batch[4].tolist()

        for index in range(batch_size):
            if 1 in dec_outputs[index]:
                dec_outputs[index] = dec_outputs[index]
            if -100 in dec_labels[index]:
                dec_labels[index] = dec_labels[index][:dec_labels[index].index(-100)]
            pred = "".join(tokenizer.convert_ids_to_tokens(dec_outputs[index][1:])).replace("Ġ", " ").replace("<pad>", "").replace("</s>", "").replace("▁", " ")
            ref = "".join(tokenizer.convert_ids_to_tokens(dec_labels[index][:-1])).replace("Ġ", " ").replace("<pad>", "").replace("</s>", "").replace("▁", " ")

            print("REFERENCE : {}\nDECODED   : {}\n".format(ref, pred))


In [ ]:

if (__name__ == "__main__"):
    save_dir = os.path.join(root_dir, "save")
    output_dir = os.path.join(root_dir, "output")
    cache_dir = os.path.join(root_dir, "cache")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)

    set_seed(seed=1234)

    config = {"mode": "train",
              "train_data_path": os.path.join(root_dir, "train.txt"),
              "test_data_path": os.path.join(root_dir, "test.txt"),
              "output_dir_path": output_dir,
              "save_dir_path": save_dir,
              "cache_dir_path": cache_dir,
              "pretrained_model_name_or_path": "hyunwoongko/kobart",
              "max_length": 250,
              "max_dec_length": 60,
              "epoch": 30,
              "batch_size": 16,
              "seed": 42,
              }

    if (config["mode"] == "train"):
        train(config)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


convert_data2feature: 100%|██████████| 14/14 [00:00<00:00, 714.47it/s]


50 Processed.. Total Loss : 0.9186775088310242
100 Processed.. Total Loss : 0.5787960290908813
150 Processed.. Total Loss : 0.6136471629142761
200 Processed.. Total Loss : 0.6417865753173828
250 Processed.. Total Loss : 0.6256996989250183
300 Processed.. Total Loss : 0.7176968455314636
350 Processed.. Total Loss : 0.5193691849708557
400 Processed.. Total Loss : 0.5036088824272156
450 Processed.. Total Loss : 0.5933317542076111
500 Processed.. Total Loss : 0.6973477602005005


100%|██████████| 1/1 [00:00<00:00,  3.78it/s]

REFERENCE :  한스킨, 직장인 대상 ‘미생(美<unk>)’ 페이스북 이벤트 실시
DECODED   :  한스킨, 페이스북 이벤트 실시

REFERENCE :  부산 도심 녹지광장 만든다
DECODED   :  부산 도심 광화문광장 조성

REFERENCE :  경제부/5. 11/2010 대한민국영어교육박람회 14일 개막..
DECODED   :  2010 대한민국영어교육박람회 14일부터 3일간 엑스코서 개최

REFERENCE :  “국정화 반대 시국선언 교사 징계”
DECODED   :  교육부, 시국선언 교사들 파악...중징계 요구

REFERENCE :  미세먼지 걱정없는 필리핀의 고급 주거단지
DECODED   :  포스코건설, 필리핀 클락 클락힐즈

REFERENCE :  한반도 상공 신호정보, 2020년까지 분석 시도
DECODED   :  신호정보 수집 분석, 2020년까지 구축

REFERENCE :  로얄앤컴퍼니, 장애인용 안전손잡이
DECODED   :  로얄앤컴퍼니 / 장애인용 안전손잡이 8종 출시

REFERENCE :  과도한 온도상승 전기찜질기 등 리콜 조치
DECODED   :  전기찜질기 4개, 전기카펫 2개 등 14개 제품 안전성 조사

REFERENCE :  임상수 감독과 마틴 스콜세지 ‘만남’
DECODED   :  임상수 감독, 마틴 스콜세지와 만남 “난 당신의 팬”

REFERENCE :  화요시조창작교실 운영
DECODED   :  대구시조시인협회, 시조창작교실 운영

REFERENCE :  주담대 금리 일제히 인상...변동형 상품 최고 4.6%
DECODED   :  변동형 주택담보대출 금리 18일부터 일제히 인상

REFERENCE :  CJ오쇼핑, 농촌 봉사활동 실시
DECODED   :  CJ오쇼핑, 배 재배농가 현명농장서 농촌 봉사활동 실시

REFERENCE :  채은정, 몽환&관능 매력 물씬
DECODED   :  채은정, 몽환적인 화보 공개...‘키니야 놀자’

REFERENCE :  설비투자 증가

550 Processed.. Total Loss : 0.5965248942375183
600 Processed.. Total Loss : 0.5923537015914917
650 Processed.. Total Loss : 0.5332625508308411
700 Processed.. Total Loss : 0.6577810049057007
750 Processed.. Total Loss : 0.6895549893379211
800 Processed.. Total Loss : 0.5265896320343018
850 Processed.. Total Loss : 0.5323483943939209
900 Processed.. Total Loss : 0.4494875371456146
950 Processed.. Total Loss : 0.5538331270217896
1000 Processed.. Total Loss : 0.4866410195827484


100%|██████████| 1/1 [00:00<00:00,  3.96it/s]

REFERENCE :  한스킨, 직장인 대상 ‘미생(美<unk>)’ 페이스북 이벤트 실시
DECODED   :  한스킨, 페이스북 이벤트 실시

REFERENCE :  부산 도심 녹지광장 만든다
DECODED   :  부산 도심 광화문광장형 도심광장

REFERENCE :  경제부/5. 11/2010 대한민국영어교육박람회 14일 개막..
DECODED   :  영어교육박람회 14일부터 대구 엑스코서 열려

REFERENCE :  “국정화 반대 시국선언 교사 징계”
DECODED   :  교육부, 시국선언 교사 징계키로

REFERENCE :  미세먼지 걱정없는 필리핀의 고급 주거단지
DECODED   :  포스코건설, 필리핀 ‘더샵 클락힐즈’

REFERENCE :  한반도 상공 신호정보, 2020년까지 분석 시도
DECODED   :  한반도 상공 신호정보 분석, 2020년까지 구축

REFERENCE :  로얄앤컴퍼니, 장애인용 안전손잡이
DECODED   :  로얄앤컴퍼니, 노약자·장애인용 안전손잡이 8종 출시

REFERENCE :  과도한 온도상승 전기찜질기 등 리콜 조치
DECODED   :  전기찜질기 등 14개 리콜명령

REFERENCE :  임상수 감독과 마틴 스콜세지 ‘만남’
DECODED   :  임상수 감독, 美영화제 초청

REFERENCE :  화요시조창작교실 운영
DECODED   :  시조 시조창작교실 운영... 9월부터 11월까지 운영

REFERENCE :  주담대 금리 일제히 인상...변동형 상품 최고 4.6%
DECODED   :  시중은행 변동형 주택담보대출 금리 일제히 오른다

REFERENCE :  CJ오쇼핑, 농촌 봉사활동 실시
DECODED   :  CJ오쇼핑, 배 재배농가 찾아 봉사활동

REFERENCE :  채은정, 몽환&관능 매력 물씬
DECODED   :  채은정, 몽환적인 화보 공개

REFERENCE :  설비투자 증가에 경기도 실물경제 회복세 유지
DECODED   :  경기 제조업 제조업 생산·설비